In [1]:
import torch 
import torch.nn as nn
import numpy as np
import pandas as pd
import yfinance as yf

In [6]:
tsla = yf.Ticker("tsla")

In [8]:
tsla = tsla.history(period='5yr', interval='5d', start='2020-01-01')
tsla

YFRateLimitError: Too Many Requests. Rate limited. Try after a while.